In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1393439701667132066
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6622735237
locality {
  bus_id: 1
  links {
  }
}
incarnation: 507513118787217036
physical_device_desc: "device: 0, name: GeForce RTX 2070 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [7]:
import os
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.datasets import mnist

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform,he_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K
from keras.utils import plot_model,normalize

from sklearn.metrics import roc_curve,roc_auc_score

In [8]:
from fv_utils import *
from model import *

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13100571898534046715
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6622735237
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10254707682710104142
physical_device_desc: "device: 0, name: GeForce RTX 2070 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [9]:
nb_classes = 140
datapath = './DB1_A/'
x_train,y_train,x_test,y_test,testfiles = get_data_label(datapath,ratio = 0.2)

In [10]:
class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        anchor, positive, negative = inputs
        p_dist = K.sum(K.square(anchor-positive), axis=-1)
        n_dist = K.sum(K.square(anchor-negative), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

def build_model(input_shape, network, margin=0.2):
    '''
    Define the Keras Model for training 
        Input : 
            input_shape : shape of input images
            network : Neural network to train outputing embeddings
            margin : minimal distance between Anchor-Positive and Anchor-Negative for the lossfunction (alpha)
    
    '''
     # Define the tensors for the three input images
    anchor_input = Input(input_shape, name="anchor_input")
    positive_input = Input(input_shape, name="positive_input")
    negative_input = Input(input_shape, name="negative_input") 
    
    # Generate the encodings (feature vectors) for the three images
    encoded_a = network(anchor_input)
    encoded_p = network(positive_input)
    encoded_n = network(negative_input)
    
    #TripletLoss Layer
    loss_layer = TripletLossLayer(alpha=margin,name='triplet_loss_layer')([encoded_a,encoded_p,encoded_n])
    
    # Connect the inputs with the outputs
    network_train = Model(inputs=[anchor_input,positive_input,negative_input],outputs=loss_layer)
    
    # return the model
    return network_train

In [12]:
input_shape=(96, 96, 3)
FRmodel = fingerRecoModel(input_shape=(96, 96, 3),embeddingsize =128)
network_train = build_model(input_shape,FRmodel)
optimizer = Adam(lr = 0.00006)
network_train.compile(loss=None,optimizer=optimizer)
network_train.summary()
plot_model(network_train,show_shapes=True, show_layer_names=True, to_file='02 model.png')
print(network_train.metrics_names)
n_iteration=0
network_train.load_weights('fv_DB1_A_weightes.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
FingerRecoModel (Model)         (None, 128)          15141568    anchor_input[0][0]               
                                                                 positive_input[0][0]             
          

In [13]:
def verify(image_path, identity, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".
    
    Arguments:
    image_path -- path to an image
    identity -- string, name of the person you'd like to verify the identity. Has to be a resident of the Happy house.
    database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
    model -- your Inception model instance in Keras
    
    Returns:
    dist -- distance between the image_path and the image of "identity" in the database.
    door_open -- True, if the door should open. False otherwise.
    """
    
    ### START CODE HERE ###
    
    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = img_to_encoding(image_path, model)
    
    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = np.linalg.norm(encoding-database[identity])
    
    # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
    if dist < 0.7:
        print("It's " + str(identity) + ", it's a match")
        door_open = True
    else:
        print("It's not " + str(identity) + ", fingerprint mismatch")
        door_open = False
        
    ### END CODE HERE ###
        
    return dist, door_open

In [14]:
database = {}
for i in y_test:
    database[i] = img_to_encoding(datapath+i+'_2.bmp',FRmodel)

In [15]:
for j in range(140):
    for i in [1,10]:
        print(verify('./DB1_A/'+y_test[j]+'_'+str(i)+'.bmp', y_test[j], database, FRmodel))

It's 100, it's a match
(0.30614078, True)
It's 100, it's a match
(0.47344637, True)
It's 101, it's a match
(0.52426803, True)
It's 101, it's a match
(0.25443122, True)
It's 102, it's a match
(0.42605072, True)
It's 102, it's a match
(0.43370348, True)
It's 103, it's a match
(0.20147836, True)
It's 103, it's a match
(0.4116692, True)
It's 104, it's a match
(0.3275344, True)
It's 104, it's a match
(0.53897923, True)
It's 105, it's a match
(0.47652578, True)
It's 105, it's a match
(0.4058526, True)
It's 106, it's a match
(0.25179067, True)
It's 106, it's a match
(0.25060672, True)
It's 107, it's a match
(0.5842305, True)
It's 107, it's a match
(0.38626713, True)
It's 108, it's a match
(0.29480216, True)
It's 108, it's a match
(0.25883847, True)
It's 109, it's a match
(0.38263527, True)
It's 109, it's a match
(0.28620607, True)
It's not 10, fingerprint mismatch
(0.8061264, False)
It's 10, it's a match
(0.5980566, True)
It's 110, it's a match
(0.4687447, True)
It's 110, it's a match
(0.4710

It's 73, it's a match
(0.26329774, True)
It's 74, it's a match
(0.33199498, True)
It's 74, it's a match
(0.33824754, True)
It's 75, it's a match
(0.416277, True)
It's 75, it's a match
(0.60593164, True)
It's 76, it's a match
(0.20735419, True)
It's 76, it's a match
(0.5426447, True)
It's 77, it's a match
(0.30768225, True)
It's 77, it's a match
(0.3753275, True)
It's 78, it's a match
(0.40995225, True)
It's 78, it's a match
(0.49602616, True)
It's 79, it's a match
(0.24525134, True)
It's 79, it's a match
(0.28448102, True)
It's 7, it's a match
(0.31021878, True)
It's 7, it's a match
(0.27102605, True)
It's 80, it's a match
(0.51572317, True)
It's 80, it's a match
(0.2305523, True)
It's 81, it's a match
(0.38639525, True)
It's 81, it's a match
(0.30693433, True)
It's 82, it's a match
(0.6167742, True)
It's 82, it's a match
(0.5599483, True)
It's not 83, fingerprint mismatch
(0.7722164, False)
It's 83, it's a match
(0.31310782, True)
It's 84, it's a match
(0.24102907, True)
It's 84, it's

In [24]:
len(database)

140

In [73]:
ball = []

In [74]:
for i in database:
    ball.append(database[i][0])

In [47]:
import miniball

In [ ]:
C,r2 = miniball.get_bounding_ball(np.array(ball))

(140, 128)

TypeError: values() takes no arguments (1 given)

(1, 128)